# ResNet vs VGG16 Transfer Learning using Keras

## Importing Libraries and Data Prep for ResNet 

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [3]:
!curl -O https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  249M  100  249M    0     0  1009k      0  0:04:12  0:04:12 --:--:-- 2227k


In [4]:
!unzip -o concrete_data_week4 -x "__MACOSX/*"

Archive:  concrete_data_week4.zip
   creating: /Users/sriram/Projects/Coursera/concrete_data_week4
   creating: /Users/sriram/Projects/Coursera/concrete_data_week4/valid
   creating: /Users/sriram/Projects/Coursera/concrete_data_week4/valid/positive
  inflating: concrete_data_week4/valid/positive/16679_1.jpg  
  inflating: concrete_data_week4/valid/positive/19463.jpg  
  inflating: concrete_data_week4/valid/positive/19041_1.jpg  
  inflating: concrete_data_week4/valid/positive/19004_1.jpg  
  inflating: concrete_data_week4/valid/positive/18041_1.jpg  
  inflating: concrete_data_week4/valid/positive/18802_1.jpg  
  inflating: concrete_data_week4/valid/positive/17679_1.jpg  
  inflating: concrete_data_week4/valid/positive/18490_1.jpg  
  inflating: concrete_data_week4/valid/positive/19477.jpg  
  inflating: concrete_data_week4/valid/positive/18004_1.jpg  
  inflating: concrete_data_week4/valid/positive/18847_1.jpg  
  inflating: concrete_data_week4/valid/positive/17111_1.jpg  
  inflatin

In [5]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [6]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [7]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [8]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


## Building ResNet Based Model

In [9]:
model = Sequential()

In [10]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [11]:
model.add(Dense(num_classes, activation='softmax'))

In [12]:
model.layers

[<Functional name=resnet50, built=True>, <Dense name=dense, built=True>]

In [14]:
model.layers[0].layers

[<InputLayer name=input_layer, built=True>,
 <ZeroPadding2D name=conv1_pad, built=True>,
 <Conv2D name=conv1_conv, built=True>,
 <BatchNormalization name=conv1_bn, built=True>,
 <Activation name=conv1_relu, built=True>,
 <ZeroPadding2D name=pool1_pad, built=True>,
 <MaxPooling2D name=pool1_pool, built=True>,
 <Conv2D name=conv2_block1_1_conv, built=True>,
 <BatchNormalization name=conv2_block1_1_bn, built=True>,
 <Activation name=conv2_block1_1_relu, built=True>,
 <Conv2D name=conv2_block1_2_conv, built=True>,
 <BatchNormalization name=conv2_block1_2_bn, built=True>,
 <Activation name=conv2_block1_2_relu, built=True>,
 <Conv2D name=conv2_block1_0_conv, built=True>,
 <Conv2D name=conv2_block1_3_conv, built=True>,
 <BatchNormalization name=conv2_block1_0_bn, built=True>,
 <BatchNormalization name=conv2_block1_3_bn, built=True>,
 <Add name=conv2_block1_add, built=True>,
 <Activation name=conv2_block1_out, built=True>,
 <Conv2D name=conv2_block2_1_conv, built=True>,
 <BatchNormalization na

In [15]:
model.layers[0].trainable = False

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,591,810 (90.00 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [19]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


301/301 ━━━━━━━━━━━━━━━━━━━━ 438s 1s/step - accuracy: 0.9728 - loss: 0.0699 - val_accuracy: 0.9982 - val_loss: 0.0065
Epoch 2/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 454s 2s/step - accuracy: 0.9990 - loss: 0.0055 - val_accuracy: 0.9987 - val_loss: 0.0046


In [20]:
model.save('classifier_resnet_model.h5')

## Importing Libraries and Data Prep for VGG16

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [22]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [23]:
data_generator = ImageDataGenerator(
    preprocessing_function = keras.applications.vgg16.preprocess_input,
)

In [24]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [25]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


## Building VGG16 Based Model

In [26]:
model = Sequential()

In [27]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [28]:
model.add(Dense(num_classes, activation='softmax'))

In [29]:
model.layers

[<Functional name=vgg16, built=True>, <Dense name=dense_1, built=True>]

In [30]:
model.layers[0].layers

[<InputLayer name=input_layer_2, built=True>,
 <Conv2D name=block1_conv1, built=True>,
 <Conv2D name=block1_conv2, built=True>,
 <MaxPooling2D name=block1_pool, built=True>,
 <Conv2D name=block2_conv1, built=True>,
 <Conv2D name=block2_conv2, built=True>,
 <MaxPooling2D name=block2_pool, built=True>,
 <Conv2D name=block3_conv1, built=True>,
 <Conv2D name=block3_conv2, built=True>,
 <Conv2D name=block3_conv3, built=True>,
 <MaxPooling2D name=block3_pool, built=True>,
 <Conv2D name=block4_conv1, built=True>,
 <Conv2D name=block4_conv2, built=True>,
 <Conv2D name=block4_conv3, built=True>,
 <MaxPooling2D name=block4_pool, built=True>,
 <Conv2D name=block5_conv1, built=True>,
 <Conv2D name=block5_conv2, built=True>,
 <Conv2D name=block5_conv3, built=True>,
 <MaxPooling2D name=block5_pool, built=True>,
 <GlobalAveragePooling2D name=global_average_pooling2d, built=True>]

In [31]:
model.layers[0].trainable = False

In [32]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 512)            │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,714 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [35]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 1484s 5s/step - accuracy: 0.9134 - loss: 0.2024 - val_accuracy: 0.9954 - val_loss: 0.0230
Epoch 2/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 1465s 5s/step - accuracy: 0.9954 - loss: 0.0201 - val_accuracy: 0.9962 - val_loss: 0.0139


In [36]:
model.save('classifier_vgg16_model.h5')

## Evaluation

In [37]:
res_model = keras.models.load_model('classifier_resnet_model.h5')

In [38]:
vgg_model = keras.models.load_model('classifier_vgg16_model.h5')

In [39]:
res_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,591,812 (90.00 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 2 (12.00 B)

In [40]:
vgg_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 512)            │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,716 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)

In [41]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    shuffle = False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.


In [42]:
# Evaluate VGG16-based model
vgg_performance = vgg_model.evaluate(test_generator, verbose=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.9983 - loss: 0.0089


In [43]:
# Evaluate ResNet50-based model
resnet50_performance = res_model.evaluate(test_generator, verbose=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 1.0000 - loss: 0.0019
